In [1]:
import numpy as np
import pandas as pd
import torch
import scipy.ndimage


In [2]:
#Normalization
def normalize_volume(volume):
    mean = np.mean(volume)
    std = np.std(volume)
    normalized_volume = (volume - mean) / std
    return normalized_volume

In [3]:
#random rotations
def random_rotation(volume, roi_coords, max_angle):
    '''
    Performs random rotations on all 3 axes. Rotation values are generated for each axis in specified range\n
    Generated by ChatGPT
    '''
    # Generate random angles for each axis within the range [-max_angle, +max_angle]
    angles = np.random.uniform(-max_angle, max_angle, size=3)
    
    # Rotate the volume with reshape=True to avoid cropping
    rotated_volume = scipy.ndimage.rotate(volume, angles[0], axes=(1, 2), reshape=True, order=1)
    rotated_volume = scipy.ndimage.rotate(rotated_volume, angles[1], axes=(0, 2), reshape=True, order=1)
    rotated_volume = scipy.ndimage.rotate(rotated_volume, angles[2], axes=(0, 1), reshape=True, order=1)
    
    # Adjust ROI coordinates according to the new shape
    def rotate_coords(coords, angle, axes, shape):
        rotation_matrix = scipy.ndimage.interpolation.rotate(np.eye(3), angle, axes=axes, reshape=False)
        new_coords = np.dot(coords - np.array(shape) / 2, rotation_matrix) + np.array(shape) / 2
        return new_coords
    
    # Rotate the ROI coordinates with the corresponding shapes after each rotation
    rotated_roi_coords = rotate_coords(roi_coords, angles[0], axes=(1, 2), shape=volume.shape)
    rotated_roi_coords = rotate_coords(rotated_roi_coords, angles[1], axes=(0, 2), shape=rotated_volume.shape)
    rotated_roi_coords = rotate_coords(rotated_roi_coords, angles[2], axes=(0, 1), shape=rotated_volume.shape)
    
    return rotated_volume, rotated_roi_coords

In [4]:
#random scaling
import numpy as np
import scipy.ndimage

def random_scaling(volume, roi_coords, max_scale=0.1):
    '''
    Performs random scaling on all 3 axes. Scale values are generated for each axis in specified range\n
    Generated by ChatGPT
    '''
    # Generate random scale factors for each axis within the range [-max_angle, +max_angle]
    scales = np.random.uniform(1 - max_scale, 1 + max_scale, size=3)
    scaled_volume = scipy.ndimage.zoom(volume, zoom=scales, order=1)

    def scale_coords(coords, scales, original_shape):
        return coords * scales + (np.array(scaled_volume.shape) - np.array(original_shape) * scales) / 2

    scaled_roi_coords = scale_coords(roi_coords, scales, volume.shape)
    
    return scaled_volume, scaled_roi_coords

In [5]:
#gaussian noise
def gaussian_noise(volume, noise_std):
    noise = np.random.normal(0, noise_std, volume.shape)
    noisy_volume = volume + noise
    return noisy_volume

In [6]:
def random_shift_and_crop(volume, roi_coords, new_shape, max_shift):
    '''
    Crops / zero-pads volume into new dimensions, and applies random shifts to the volume relative to the center of the new shape\n
    Generated by ChatGPT
    '''
    assert len(new_shape) == 3, "new_shape must be a tuple of three dimensions (depth, height, width)"
    
    # Get current volume shape
    current_shape = volume.shape
    
    # Compute center indices
    current_center = np.array(current_shape) // 2
    new_center = np.array(new_shape) // 2
    
    # Generate random shifts within max_shift
    shifts = np.random.randint(-max_shift, max_shift + 1, size=3)
    
    # Calculate start and end indices for cropping
    start_idx = current_center - new_center + shifts
    
    # Initialize cropped volume and mask
    cropped_volume = np.zeros(new_shape)
    
    # Calculate valid crop range within current volume
    crop_start = np.maximum(-start_idx, 0)
    crop_end = np.minimum(current_shape - start_idx, new_shape)
    
    # Copy cropped region from original volume to cropped volume
    cropped_volume[crop_start[0]:crop_end[0],
                   crop_start[1]:crop_end[1],
                   crop_start[2]:crop_end[2]] = volume[start_idx[0]+crop_start[0]:start_idx[0]+crop_end[0],
                                                         start_idx[1]+crop_start[1]:start_idx[1]+crop_end[1],
                                                         start_idx[2]+crop_start[2]:start_idx[2]+crop_end[2]]
    
    # Adjust ROI coordinates based on cropping and shifts
    shifted_roi_coords = roi_coords - start_idx
    
    # Create mask for coordinates outside new shape
    outside_mask = (shifted_roi_coords[:, 0] < 0) | (shifted_roi_coords[:, 0] >= new_shape[0]) | \
                   (shifted_roi_coords[:, 1] < 0) | (shifted_roi_coords[:, 1] >= new_shape[1]) | \
                   (shifted_roi_coords[:, 2] < 0) | (shifted_roi_coords[:, 2] >= new_shape[2])
    
    return cropped_volume, shifted_roi_coords, outside_mask